In [ ]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, img_ch):
        super(Discriminator, self).__init__()
        self.discriminate = nn.Sequential(
            nn.Conv2d(img_ch, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.discriminate(input).view(-1)


class Generator(nn.Module):
    def __init__(self, in_channel, img_ch):
        super(Generator, self).__init__()
        self.generate = nn.Sequential(
            nn.ConvTranspose2d(in_channel, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_ch, 4, 2, 1, bias=False),
            nn.Tanh()
        )
  
    def forward(self, input):
        return self.generate(input)



In [ ]:
!unzip DogFaceNet_Dataset_224_1 -d 'dog_data'

串流輸出內容已截斷至最後 5000 行。
  inflating: dog_data/after_4_bis/326/326.3.jpg  
  inflating: dog_data/after_4_bis/326/326.4.jpg  
  inflating: dog_data/after_4_bis/326/326.5.jpg  
  inflating: dog_data/after_4_bis/326/326.6.jpg  
  inflating: dog_data/after_4_bis/326/326.7.jpg  
   creating: dog_data/after_4_bis/327/
  inflating: dog_data/after_4_bis/327/327.0.jpg  
  inflating: dog_data/after_4_bis/327/327.1.jpg  
  inflating: dog_data/after_4_bis/327/327.2.jpg  
  inflating: dog_data/after_4_bis/327/327.3.jpg  
  inflating: dog_data/after_4_bis/327/327.4.jpg  
  inflating: dog_data/after_4_bis/327/327.5.jpg  
   creating: dog_data/after_4_bis/328/
  inflating: dog_data/after_4_bis/328/328.0.jpg  
  inflating: dog_data/after_4_bis/328/328.1.jpg  
  inflating: dog_data/after_4_bis/328/328.2.jpg  
  inflating: dog_data/after_4_bis/328/328.3.jpg  
  inflating: dog_data/after_4_bis/328/328.4.jpg  
  inflating: dog_data/after_4_bis/328/328.5.jpg  
  inflating: dog_data/after_4_bis/328/328.6.jpg  
 

In [ ]:
import os
import numpy as np
from PIL import Image

import torch.utils.data as data
import torchvision.transforms as transforms

class DogFace(data.Dataset):
  def __init__(self, root='dog_data/after_4_bis'):
    self.root = root
    self.img_paths = []

    self.transform = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(), 
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])

    files = os.listdir(self.root)
    for file in files:
      path = os.path.join(self.root, file)
      images = os.listdir(path)
      for image in images:
        img_path = os.path.join(path, image)
        self.img_paths.append(img_path)

  def __len__(self):
    return len(self.img_paths)
  def __getitem__(self, index):
    img_path = self.img_paths[index]
    img = Image.open(img_path).convert('RGB')
    img = self.transform(img)

    return img


In [ ]:
import time
import matplotlib.pyplot as plt

import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F

class Network():
    def __init__(self, latent_dims=64, num_epochs=50, batch_size=64, learning_rate=2.0e-4, dataset='mnist', gpu=True):
        self.latent_dims = latent_dims
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.in_channel = 100
        self.learning_rate = learning_rate
        self.gpu = gpu
        self.dur_time = 0

        if dataset == 'mnist' or dataset == 'fashion_mnist':
            self.img_ch = 1
        else:
            self.img_ch = 3

        self._init_dataset()
        self._init_model()

    def _init_dataset(self):

        train_data = DogFace()
        self.train_queue = data.DataLoader(train_data, batch_size=self.batch_size, shuffle=True, drop_last=True)
        
    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)

    def _init_model(self):
        self.device = torch.device('cuda' if (torch.cuda.is_available() & self.gpu) else 'cpu')

        Gen = Generator(self.in_channel, self.img_ch).to(self.device)
        Dis = Discriminator(self.img_ch).to(self.device)
        self.Gen = Gen.to(self.device)
        self.Dis = Dis.to(self.device)
        
        self.Gen.apply(self.weights_init)
        self.Dis.apply(self.weights_init)
        
        self.criterion = nn.BCELoss()

        #self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.learning_rate, weight_decay=1e-5)
        self.G_optimizer = optim.Adam(self.Gen.parameters(), lr=self.learning_rate, betas=(0.5, 0.999))    # optimizer
        self.D_optimizer = optim.Adam(self.Dis.parameters(), lr=self.learning_rate, betas=(0.5, 0.999))

    def show_images(self, images, num_images=16):
        #sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
        images = images.permute(0, 2, 3, 1)
        images = np.array(images)
        #print(images)  
        sqrtn = int(np.ceil(np.sqrt(num_images)))

        for index, image in enumerate(images):
            if index < num_images:
                #print(image.shape)
                plt.subplot(sqrtn, sqrtn, index+1)    
                plt.imshow(image)
                plt.axis('off')


    def calc_time(self, seconds):
        m, s = divmod(seconds, 60)
        h, m = divmod(m, 60)
        t, h = divmod(h, 24)
        return {'day':t, 'hour':h, 'minute':m, 'second':int(s)}
    

    def train_dog(self):
        G_loss_list = []
        D_loss_list = []
        run_start = time.time()

        self.Gen.train()
        self.Dis.train()
        for epoch in range(self.num_epochs):
            G_loss_sum = 0
            D_loss_sum = 0

            show_inputs = None
            show_outputs = None
            for step, inputs in enumerate(self.train_queue):
                # Train Disciminator For Real Image
                self.Dis.zero_grad()
                #print(inputs.shape)
                inputs_real = inputs.to(self.device) # 64,1,28,28
                real_label = torch.ones(self.batch_size,).to(self.device) # 64個1，代表真實的圖片
                inputs_real_predict = self.Dis(inputs_real)

                #print(inputs_real_predict.shape, real_label.shape)
                D_real_loss = self.criterion(inputs_real_predict, real_label) # view make [64,1,1,1] -> [64] and cal loss
                D_real_loss.backward()

                # Train Disciminator For Fake Image
                noise = torch.tensor(torch.randn(self.batch_size, self.in_channel, 1, 1, device=self.device)) # torch.Size([64, 100, 1, 1])
                fake_label = torch.zeros(self.batch_size,).to(self.device)   # 64個0，代表假的圖片
                fake_out = self.Gen(noise)                   # 64,1,28,28
                fake_predict = self.Dis(fake_out.detach())           # 64,1,1,1
                
                #print(fake_predict.shape, fake_label.shape)
                D_fake_loss = self.criterion(fake_predict, fake_label) # view make [64,1,1,1] -> [64] and cal loss
                D_fake_loss.backward()

                D_total_loss = D_real_loss + D_fake_loss
                D_loss_list.append(D_total_loss.item())
                D_loss_sum += D_total_loss
                self.D_optimizer.step()

                self.Gen.zero_grad()

                # 訓練生成真實圖片
                #noise = torch.tensor(torch.randn(self.batch_size, self.in_channel, 1, 1, device=self.device)) # torch.Size([64, 100, 1, 1])
                target_label = torch.ones(self.batch_size,).to(self.device) # 64個1，代表希望產生的真實圖片
                #fake_out2 = self.Gen(noise)      # 生成假圖片
                fake_predict2 = self.Dis(fake_out)  # 獲得Discriminator的判斷

                G_loss = self.criterion(fake_predict2, target_label)
                G_loss_list.append(G_loss.item())
                G_loss_sum += G_loss
                G_loss.backward()
                self.G_optimizer.step()

                show_outputs = fake_out.detach().cpu()

            trainG_avg_loss = G_loss_sum/step
            trainD_avg_loss = D_loss_sum/step
            print('Epoch [{}]/[{}], D Loss:{}, G Loss:{}'.format(epoch, self.num_epochs, trainD_avg_loss, trainG_avg_loss))
            print('cost time: {}'.format(self.calc_time(self.dur_time + time.time() - run_start)))

            #if epoch == self.num_epochs-1:
            #self.show_images(show_inputs)
            #plt.savefig('inputs_fashion_'+str(self.latent_dims)+'.png')
            #plt.show()
            self.show_images(show_outputs)
            plt.savefig('outputs_dogFace.png')
            plt.show()

        plt.subplot(211)
        plt.plot(G_loss_list)
        plt.subplot(212)
        plt.plot(D_loss_list)

        torch.save(self.Gen, './DCGAN_Generator_dogFace.pth')
        torch.save(self.Dis, './DCGAN_Discriminator_dogFace.pth')

In [ ]:
if __name__ == '__main__':
    train_network = Network(dataset='dogface')
    train_network.train_dog()

Output hidden; open in https://colab.research.google.com to view.